In [ ]:
!pip install google-cloud-storage


In [1]:
import sys
import os

# Get the absolute path of the 'app' folder
sys.path.append(os.path.abspath('../app'))

# Now import Frame from frame.py
from frame import Frame

# Create an instance and call dataset()
data = Frame()
df = data.dataset()


Fetching and Merging...
Merging completed successfully!
Started prepocessing....


/home/sb/lewagon_london/project_lewagon/stock-stat-replica/app/frame.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_latest.drop(columns='HLN',inplace=True)
/home/sb/lewagon_london/project_lewagon/stock-stat-replica/app/frame.py:59: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_latest=df_latest.fillna(method='ffill')


Completed prepocessing.
Fetching and Merging...
Merging completed successfully!
Started prepocessing....
Completed prepocessing.


/home/sb/lewagon_london/project_lewagon/stock-stat-replica/app/frame.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_latest.drop(columns='HLN',inplace=True)
/home/sb/lewagon_london/project_lewagon/stock-stat-replica/app/frame.py:59: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_latest=df_latest.fillna(method='ffill')


In [3]:
# df['HLN']

In [ ]:
df.isna().sum().sort_values(ascending=True)

In [ ]:
df['date'].value_counts(ascending=False)

In [ ]:
from google.cloud import storage
import pandas as pd
import io
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [ ]:
from google.colab import files

uploaded = files.upload()  #upload credentials file

In [ ]:
filename = list(uploaded.keys())[0]
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/" + filename

In [ ]:
client = storage.Client()
bucket = client.get_bucket('stat_arb')


In [ ]:
rename_dict = {
                    'Unnamed: 0': 'date',
                    '1. open': 'open',
                    '2. high': 'high',
                    '3. low': 'low',
                    '4. close': 'close',
                    '5. volume': 'volume'
                }

In [ ]:
folder_prefix = "FTSE_100/"
blobs = bucket.list_blobs(prefix=folder_prefix)
dataframes = []

for blob in blobs:
    if blob.name.endswith('.csv'):
        content = blob.download_as_string()
        if content.strip():
            try:
                df = pd.read_csv(io.StringIO(content.decode('utf-8')), sep=None, engine='python')

                df = df.rename(columns=rename_dict)
                expected_columns = rename_dict.values()
                df = df[[col for col in df.columns if col in expected_columns]]
                stk=blob.name.split('/')[1]
                df['source_file']=stk.split('.')[0]
                dataframes.append(df)

            except pd.errors.EmptyDataError:
                print(f"Skipping empty file: {blob.name}")
        else:
            print(f"Skipping empty file: {blob.name}")

if dataframes:
    merged_df = pd.concat(dataframes, ignore_index=True)
    print("Merging completed successfully!")
else:
    print("No valid CSV files found.")

merged_df

In [ ]:
merged_df['date'] = pd.to_datetime(merged_df['date'],format='mixed')


In [ ]:
df_modified = merged_df[['date', 'source_file', 'open']].copy()
df_modified['source_file'] = df_modified['source_file'].str.split('/').str[-1].str.replace('.csv', '')
df_pivoted = df_modified.pivot_table(index='date', columns='source_file', values='open')
df_pivoted.reset_index(inplace=True)

df_pivoted

In [ ]:
df_pivoted[df_pivoted['AAF'].isna()]

In [ ]:
df_latest=df_pivoted[df_pivoted['date']>'2022-01-31']

In [ ]:
df_latest.isna().sum()

In [ ]:
# df_latest.dropna(inplace=True)

In [ ]:
df_numeric = df_latest.drop(columns=['date'])


In [ ]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_numeric)

In [ ]:
pca = PCA(n_components=0.95)
df_pca = pca.fit_transform(df_scaled)

In [ ]:
df_pca = pd.DataFrame(df_pca, columns=[f'PC{i+1}' for i in range(df_pca.shape[1])])


In [ ]:
df_pca['date'] = df_latest['date'].values


In [ ]:
df_pca

In [ ]:
print("Explained Variance Ratio:", pca.explained_variance_ratio_)
